<a href="https://colab.research.google.com/github/sid-ak/jphomevalue/blob/CD-33-Forecast-Model-Component/core/shiller_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend DB Integration

In [ ]:
!pip install firebase-admin

In [ ]:
# Imports
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Firebase Config
These creds won't work.
You can find your creds in the following link. <br>
https://drive.google.com/drive/u/0/folders/1PcGWxfJ4TguFFfV4L0MstTbn2uAAjb_o <br>
This just serves as an example for the path. <br>



In [ ]:
# Get ceredentials through service account.
cred = credentials.Certificate("/content/drive/MyDrive/jphomevalue/jphomevalue-firebase-adminsdk-g1kd5-dcefd13eb1.json")
# Initialize the app with a service account, granting admin privileges.
firebase_admin.initialize_app(cred)

# Create Ref DB

In [ ]:
ref = db.reference(url='https://jphomevalue-default-rtdb.firebaseio.com/')
ref.get()

[{'DATE': '2002-01-01', 'TPXRSA': 120.39579298067301},
 {'DATE': '2002-02-01', 'TPXRSA': 120.70003351737302},
 {'DATE': '2002-03-01', 'TPXRSA': 122.958146804667},
 {'DATE': '2002-04-01', 'TPXRSA': 123.707047327969},
 {'DATE': '2002-05-01', 'TPXRSA': 124.950321418552},
 {'DATE': '2002-06-01', 'TPXRSA': 125.597677843222},
 {'DATE': '2002-07-01', 'TPXRSA': 126.853260756778},
 {'DATE': '2002-08-01', 'TPXRSA': 127.753579001549},
 {'DATE': '2002-09-01', 'TPXRSA': 128.573840166495},
 {'DATE': '2002-10-01', 'TPXRSA': 129.20078065653},
 {'DATE': '2002-11-01', 'TPXRSA': 130.256535719701},
 {'DATE': '2002-12-01', 'TPXRSA': 131.830378013638},
 {'DATE': '2003-01-01', 'TPXRSA': 133.271389035007},
 {'DATE': '2003-02-01', 'TPXRSA': 134.661895908838},
 {'DATE': '2003-03-01', 'TPXRSA': 135.520304973853},
 {'DATE': '2003-04-01', 'TPXRSA': 136.626738169342},
 {'DATE': '2003-05-01', 'TPXRSA': 138.117429597589},
 {'DATE': '2003-06-01', 'TPXRSA': 138.665744347527},
 {'DATE': '2003-07-01', 'TPXRSA': 139.35188

# Import and Parse the Shiller Index

Imported Data: https://github.com/sid-ak/jphomevalue/blob/main/data/shiller/TPXRSA.csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas.core.indexes.multi import MultiIndex
from google.colab import files

# Read and parse the Shiller index while setting the index as the DATE.
shiller_filePath = "/content/drive/MyDrive/jphomevalue/data/shiller/TPXRSA.csv"
shiller_data = pd.read_csv(shiller_filePath,
                           parse_dates = ['DATE'])

# Preserve numerical index for future use
shiller_numberedIndex = shiller_data.index.values

# Convert index to dataframe.
shiller_df = pd.DataFrame(shiller_data)
shiller_df = shiller_df.set_index('DATE')

# Convert to monthly periods.
# shiller_df.index = shiller_df.index.to_period("1M")

# Plot index.
# Plot does not work for monthly periods.
plt.plot(shiller_df)

# Downward Trend

In [ ]:
# Get downward trend ranges.
downward_df = shiller_df.loc['2006-07-01' : '2012-01-01']

# Set serialized index for downward_df
downward_df = downward_df.set_index(pd.Index(
    np.arange(len(downward_df.index))), append=True)

# Swap the date and s_no
downward_indexList = list(map(lambda swap: (swap[1], swap[0]), downward_df.index))

# Convert tuples back to index.
downward_df.index = MultiIndex.from_tuples(downward_indexList, names=["s_no", "DATE"])

downward_df

In [ ]:
# Plot downward trend.
# plt.plot(downward_df)

# ATTENTION: Plotting this will no longer produce a graph because we
#            are resetting the index to a MultiIndex type for s_no.

# This is only for visualisation.

# Upward Trend

In [ ]:
# Get upward trend ranges.
upward_df = shiller_df.loc['2012-02-01' : '2020-12-01']

# Set serialized index for upward_df.
upward_df = upward_df.set_index(pd.Index(
    np.arange(len(upward_df.index))), append=True)

# Swap the date and s_no.
upward_sNo = list(map(lambda swap: (swap[1], swap[0]), upward_df.index))

# Convert tuples back to index.
upward_df.index = MultiIndex.from_tuples(upward_sNo, names=["s_no", "DATE"])

upward_df

In [ ]:
# Plot upward trend.
# plt.plot(upward_df)

# ATTENTION: Plotting this will no longer produce a graph because we
#            are resetting the index to a MultiIndex type for s_no.

# This is only for visualisation.

# Export to CSV and Download

In [ ]:
downward_df.to_csv('shiller_downward.csv')
upward_df.to_csv('shiller_upward.csv')

files.download('shiller_downward.csv')
files.download('shiller_upward.csv')